In [1]:
# SETUP THE MONGO_CONNECTION
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
#!pip install ollama
import ollama
ollama.pull('llama3.1')
mongo_uri = os.getenv('MONGO_URI')
# SETUP THE MONGO_CONNECTION
uri = mongo_uri if mongo_uri else "mongodb://localhost:27017"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [2]:
db = client["posts"]
posts_collection = db["posts"]


In [3]:
pending_posts = list(posts_collection.find({"status":"pending"}))
print(len(pending_posts))

227


In [4]:
PROMT1 = "You are my social media manager. I will provide you with a post, and you must tell me if it aligns with my interests. Response: You can only reply with either 'yes' or 'no'. Do not include any additional words or sentences. My Interest: I am seeking new opportunities in software development roles, having graduated in 2024 with around 1 year of experience as a full-stack developer (front-end and back-end). I am also interested in roles such as software engineer, web developer, backend engineer, frontend developer, and full-stack engineer. Posts mentioning hiring opportunities, offering referrals, or related to these roles are of the highest interest to me."
PROMT2 = "You are my social media manager , i will give you a post and you have to tell me if that post is good for my interest or not. RESPONSE: you can only respond in one word with either 'yes' or 'no', DO NOT ADD ANY OTHER SENTENCE TO IT,  MY INTEREST : I am looking for new opportunites in the software developer and realated  roles,  i passed out in 2024 and i have around 1 year of experience , I am working as a full stack developer and can work on both frontends and back ends, any post which mentions an hiring opportunity or providing reffral are my of AT MOST INTREST"
def isRelevant( query):
    if not query :
        return False;
    stream = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content': PROMT2
        },
        {
            'role': 'user',
            'content': "This is this post" + query,
        }]
    )
    stream  = stream['message']['content']
    response =""
    isQualified = stream.lower().find("yes") != -1
    print("BOT: ", stream)
    return isQualified;
    


    # print( "\n Chat line ", chatLine);
    # print( "\n Chat History", chatHistory)

In [5]:
for post in pending_posts:
    isQualified = isRelevant(post["feed_text"])
    if isQualified :
        post["status"] = "qualified"
        print(post["feed_text"])
    else :
        post["status"] = "unqualified"
    posts_collection.replace_one({"_id": post["_id"]}, post)

BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  No
BOT:  no
BOT:  Yes

 Opportunity Alert 🚀 We're hiring for the following roles at Attentive.ai.Software Engineer I - https://lnkd.in/gjUk4va9Software Engineer II  - https://lnkd.in/gcWY4HaTSoftware Engineer III - https://lnkd.in/gheqegnjLead Software Engineer - https://lnkd.in/gUKp8gAkFeel free to apply with the links attached to each job or ping me directly!

BOT:  no
BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  No
BOT:  no
BOT:  No
BOT:  No
BOT:  no
BOT:  No
BOT:  Yes

 𝐣𝐮𝐧𝐢𝐨𝐫 𝐬𝐨𝐟𝐭𝐰𝐚𝐫𝐞 𝐝𝐞𝐯𝐞𝐥𝐨𝐩𝐞𝐫 𝐢𝐧 𝐭𝐡𝐞 𝐬𝐭𝐚𝐫𝐭𝐮𝐩 𝐜𝐨𝐦𝐩𝐚𝐧𝐲...That's exciting! As a junior software developer in a startup, you'll be part of a fast-paced environment where you can learn a lot and make a significant impact. Here are some tips to help you thrive in your role:1.Embrace Learning: Startups often require you to wear many hats, so take this as an opportunity to learn about different aspects of software development and the business.2 Ask Questions: Don't be afraid to ask questio

In [6]:
qualified_posts = list(filter(lambda post: post["status"] == "qualified", pending_posts))

In [7]:
for post in qualified_posts:
    print(post["poster"] +  " :  "  + post["poster_bio"]) 
    print("profile" , post["profile"]);
    print("feed_text" , post["feed_text"])  if post else post
    print("href",post["href_map"])
    print("\n")

Sanket S. :  Engineering@Attentive | SAAS | Ex-Tally | Skilled in System Design & Complex System Architecture
profile https://www.linkedin.com/in/sanketsingh96?miniProfileUrn=urn%3Ali%3Afsd_profile%3AACoAAB6dOPQBpSwnMgXmi2yHd67_-Ek0BWlJkXk
feed_text 
 Opportunity Alert 🚀 We're hiring for the following roles at Attentive.ai.Software Engineer I - https://lnkd.in/gjUk4va9Software Engineer II  - https://lnkd.in/gcWY4HaTSoftware Engineer III - https://lnkd.in/gheqegnjLead Software Engineer - https://lnkd.in/gUKp8gAkFeel free to apply with the links attached to each job or ping me directly!

href {'Attentive.ai': 'https://www.linkedin.com/company/attentive.ai/', 'https://lnkd.in/gjUk4va9': 'https://lnkd.in/gjUk4va9', 'https://lnkd.in/gcWY4HaT': 'https://lnkd.in/gcWY4HaT', 'https://lnkd.in/gheqegnj': 'https://lnkd.in/gheqegnj', 'https://lnkd.in/gUKp8gAk': 'https://lnkd.in/gUKp8gAk'}


JavaScript :  Akshita Shrivastava • 2nd
profile https://www.linkedin.com/groups/121615?q=highlightedFeedForGr

In [8]:
len(qualified_posts)

32